<a href="https://colab.research.google.com/github/hanbiphyun/ESSA_YB/blob/main/ESAA_YB_week10_%EC%88%98%EC%83%81%EC%9E%91%EB%A6%AC%EB%B7%B0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

주소 : https://www.kaggle.com/competitions/ventilator-pressure-prediction
###Google Brain - Ventilator Pressure Prediction

코드 주소 : 1. https://www.kaggle.com/code/shujun717/1-solution-lstm-cnn-transformer-1-fold
2. https://www.kaggle.com/code/group16/1-solution-pid-controller-matching-v1

주제 :기계 인공호흡기가 환자의 폐에 연결되어있을 때의 기계 내부 압력 예측
- 폐의 특성 (순응성, 기도저항성 등)이 각각 다른 호흡 시계열 데이터 주어짐
- 이 데이터로 각 호흡 과정의 특정 시점에서의 공기압력예측

**문제유형** : 회귀 / 시간에 따른 압력 예측

데이터 형태 : 시계열 데이터

변수
- u_in : 흡입 밸브 제어 입력 (0-100)
- u_out : 호기 밸브 제어 입력 (0/1)
- R : 폐 저항성
- C : 폐 순응성
- pressure : 목표변수/ 기계회로 내 공기압력

적용배경 : 인공호흡기 제어가 중요한 시기, 결핵. 코로나19 등 인공호흡기 수요 급증

1. 예측 대상 및 평가지표
- 예측 대상 : 호흡의 여러 시점 중 흡입 단계의 입력값, 평가지표 : MAE (평균절대오차)

2. 피처엔지니어링
- 이전 시점의 u_in 값 추가 / u_in 의 변화값 (diff) 추가 -> 시계열적 변화 정보 활용
- R, C의 조합을 원-핫 인코딩하거나, u_in을 누적적분한 값 만듬-> 폐 속성, 제어 입력누적효과 반영

3. 모델구조
- LSTM -> 1D CNN -> Transformer encoder 구조 결합 모델
> LSTM : long short term memory, 인공신경망 구조 (기존 RNN의 장기 의존성 문제 해결)

-> LSTM 계층을 통해 시간 순서의 이전 상태가 현재 상태에 주는 영향 학습 (이 데이터에서 타깃 변수인 압력이 이전 제어 압력, 이전 압력에 의존하기에 유리함)

-> 1D convolution계층 사용해 인접 시점간의 지역적 상관관계를 효과적으로 관찰 (짧은 시간 간격 내 제어 입력 변화가 압력에 미치는 영향 확인)

-> Transformer encoder블록 사용해 더 넓은 범위의 시점들 간의 의존관계 학습

4. 학습/ 검증 전략
- K-fold CV 사용
- pseudo-labeling 기법 사용 => 테스트 데이터의 예측값을 다시 학습에 확용하는 반지도학습 방식
- 후처리, 앙상블 활용

**인상적인 점 및 특징 **
- 여러 모델 구조를 이용하여 데이터 관계 살핌 (LSTM + CNN + Transformer)
- 잔차 연결 적용 : LSTM의 기울기 소실 문제를 해결하기 위해 ResidualLSTM모듈을 도입
- 피처 엔지니어링 설계가 깊이있고 성능향상에 기여한 것으로 보임
- 검증과 앙상블 : 여러 전략으로 일반화 성능 높임

+ 유의점 : 구조가 복잡해 과적합, 학습시간이 길 위험이 있음, 다양한 피처엔지니어링을 하여 새로운 제어입력 패턴이 생기면 모델이 현재 학습한 특징을 덜 활용함+ 적응력 부족